# 1.1 Installing The Rust Toolchain

Installing The Rust Toolchain
- https://rustup.rs/

In [ ]:
! curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh

Release Channels
- stable
- beta
- nightly

What Toolchains Do We Need?
- `rustup update`: update your toolchains
- `rustup toolchain list`: give you an overview of what is installed on your system

In [2]:
! rustup update

info: syncing channel updates for 'stable-x86_64-unknown-linux-gnu'
info: checking for self-update

  stable-x86_64-unknown-linux-gnu unchanged - rustc 1.81.0 (eeb90cda1 2024-09-04)

info: cleaning up downloads & tmp directories


In [1]:
! rustup toolchain list

stable-x86_64-unknown-linux-gnu (default)


# 1.2 Project Setup

- `rustc`: the Rust compiler itself

In [3]:
! rustc --version

rustc 1.81.0 (eeb90cda1 2024-09-04)


- `cargo` : Rust’s build tool

In [4]:
! cargo --version

cargo 1.81.0 (2dbb1af80 2024-08-20)


use `cargo` to create the skeleton of the project we will be working on for the whole book:

In [1]:
! cargo new zero2prod

    Creating binary (application) `zero2prod` package
note: see more `Cargo.toml` keys and their definitions at https://doc.rust-lang.org/cargo/reference/manifest.html


In [2]:
! tree

.
├── 01_Getting_Started.ipynb
├── LICENSE
├── README.md
└── zero2prod
    ├── Cargo.toml
    └── src
        └── main.rs

2 directories, 5 files


- git: Github Action

> https://docs.github.com/en/authentication/connecting-to-github-with-ssh

host the project on GitHub

In [5]:
! git add .

In [7]:
! git commit -am "Project skeleton"

Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: empty ident name (for <renew2049@outlook.com>) not allowed


In [ ]:
! git remote add origin 

In [ ]:
! git@github.com:ningliu2024/zero2prod.gitgit push -u origin main